In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = []  # remove any existing handlers
file_handler = logging.FileHandler('scs.log')
file_handler.setFormatter(logging.Formatter(
    '%(name)s:%(levelname)s %(message)s'))
logger.addHandler(file_handler)

In [1]:
#Baseline experiment
import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Standard Conv Sigmoid for 10 epochs')
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, conv_reshape = True )
valid_dp = MNISTDataProvider(dset='valid', batch_size=1000, max_num_batches=-10, randomize=False, conv_reshape = True)
test_dp = MNISTDataProvider(dset='eval', batch_size=1000, max_num_batches=-10, randomize=False, conv_reshape = True)

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.conv import ConvSigmoid, ConvMaxPool2D
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
learning_rate = 0.5
max_epochs = 10
cost = CECost()
    
stats = []

train_dp.reset()
valid_dp.reset()
test_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(ConvSigmoid(num_inp_feat_maps = 1, num_out_feat_maps = 2, irange=0.2, rng=rng))
model.add_layer(ConvMaxPool2D(num_feat_maps = 2, conv_shape = (24,24)))
model.add_layer(Sigmoid(idim=2*12*12, odim=2*12*12, irange=0.2, rng=rng))
model.add_layer(Softmax(idim=2*12*12, odim=10, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

logger.info('Testing the model on test set:')
tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

shelve_r['results'] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

shelve_r.close()

INFO:root:Standard Conv Sigmoid for 10 epochs
INFO:root:Initialising data providers...


ImportError: No module named convc

In [3]:
with open('scs.log', 'r') as log_file:
    print log_file.read()

root:INFO Standard Conv Sigmoid for 10 epochs
root:INFO Initialising data providers...
root:INFO Training started...
root:INFO Standard Conv Sigmoid for 10 epochs
root:INFO Initialising data providers...
root:INFO Training started...
mlp.optimisers:INFO Epoch 0: Training cost (ce) for initial model is 2.378. Accuracy is 10.60%
mlp.optimisers:INFO Epoch 0: Validation cost (ce) for initial model is 2.397. Accuracy is 9.91%
mlp.optimisers:INFO Epoch 1: Training cost (ce) is 2.837. Accuracy is 8.50%
mlp.optimisers:INFO Epoch 1: Validation cost (ce) is 2.321. Accuracy is 9.90%
mlp.optimisers:INFO Epoch 1: Took 298 seconds. Training speed 23 pps. Validation speed 39 pps.
mlp.optimisers:INFO Epoch 2: Training cost (ce) is 2.321. Accuracy is 11.00%
mlp.optimisers:INFO Epoch 2: Validation cost (ce) is 2.306. Accuracy is 9.90%
mlp.optimisers:INFO Epoch 2: Took 298 seconds. Training speed 23 pps. Validation speed 39 pps.
mlp.optimisers:INFO Epoch 3: Training cost (ce) is 2.305. Accuracy is 12.20%